In [1]:
from utils import LoginCredentials
import wandb

authenticator = LoginCredentials()

wandb.login(key=authenticator.wandb_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: finnhenri-smidt. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/henrismidt/.netrc


True

In [2]:
import torch
from torchvision import transforms
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from transformers import MobileViTImageProcessor
import wandb

from dataset import MRIFeatureDataModule, MRIFeatureDataset
from models import SimpleEnsembleModel, AdvancedEnsembleModel, MediumEnsembleModel
from utils import get_best_device, LoginCredentials

from datetime import datetime
import lightning.pytorch as pl
import torch
import numpy as np
import random
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix


wandb.finish()  # Make sure previous sessions are finished


def set_reproducibility(seed=42):
    # Set Python random seed
    random.seed(seed)

    # Set Numpy seed
    np.random.seed(seed)

    # Set PyTorch seed
    torch.manual_seed(seed)

    # If using CUDA:
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Control sources of nondeterminism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # PyTorch Lightning utility to seed everything
    pl.seed_everything(seed, workers=True)



# Define sweep configuration
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "model_name": {"values": ["MobileVit-s", "efficientnet-b2"]},
        "ensemble_variant": {"values": ["simple", "medium", "advanced"]},
        "learning_rate": {"values": [0.005, 0.001, 0.0005, 0.0001, 0.00005]},
        "batch_size": {"values": [160]},
        "epochs": {"values": [15]},
        "random_seed": {"values": [i for i in range(10)]},
    },
}

sweep_id = wandb.sweep(sweep=sweep_config, project="Alzheimer-Detection")

# Define the training function


def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = get_best_device()
        feature_folder = "extracted_features/student/"
        model_name = config.model_name
        train_pkl = f'{feature_folder}{model_name}_train_features_pooled.pkl'
        val_pkl = f'{feature_folder}{model_name}_val_features_pooled.pkl'
        test_pkl = f'{feature_folder}{model_name}_test_features_pooled.pkl'
        num_classes = 4 
        as_sequence = config.ensemble_variant == "advanced"
        set_reproducibility(config.random_seed)
        
        data_module = MRIFeatureDataModule(train_pkl=train_pkl, val_pkl=val_pkl, test_pkl=test_pkl, as_sequence=as_sequence, batch_size=config.batch_size)
        data_module.setup()
        train_loader = data_module.train_dataloader()
        val_loader = data_module.val_dataloader()
        test_loader = data_module.test_dataloader()
        featuremap_length = data_module.featuremap_length
        sequence_length = data_module.sequence_length

        if config.ensemble_variant == "simple":
            feature_size = featuremap_length * sequence_length
            model = SimpleEnsembleModel(feature_size=feature_size, num_classes=num_classes, lr=config.learning_rate)
        elif config.ensemble_variant == "medium":
            feature_size = featuremap_length * sequence_length
            model = MediumEnsembleModel(feature_size=feature_size, num_classes=num_classes, lr=config.learning_rate)
        elif config.ensemble_variant == "advanced":
            feature_size = featuremap_length
            model = AdvancedEnsembleModel(feature_size=feature_size, num_classes=num_classes, lr=config.learning_rate, max_seq_length=sequence_length)

        wandb_logger = WandbLogger()

        checkpoint_callback = ModelCheckpoint(
            dirpath=f"model_checkpoints/ensemble_{config.ensemble_variant}_{config.model_name}",
            filename=f"lr_{config.learning_rate}",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )

        trainer = L.Trainer(
            max_epochs=config.epochs,
            devices="auto",
            accelerator="auto",
            logger=wandb_logger,
            callbacks=[checkpoint_callback],
            log_every_n_steps=config.batch_size//2,
        )

        trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        # Load best model for testing
        best_model_path = checkpoint_callback.best_model_path
        if config.ensemble_variant == "simple":
            best_model_class = SimpleEnsembleModel
            feature_size = featuremap_length * sequence_length
        elif config.ensemble_variant == "medium":
            best_model_class = MediumEnsembleModel
            feature_size = featuremap_length * sequence_length
        elif config.ensemble_variant == "advanced":
            best_model_class = AdvancedEnsembleModel
            feature_size = featuremap_length

        best_model = best_model_class.load_from_checkpoint(
            best_model_path,
            feature_size=feature_size,
            num_classes=num_classes,
            lr=config.learning_rate,
            max_seq_length=sequence_length 
        )
        best_model = best_model.to(device)
        best_model.eval()

        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_loader:
                inputs, labels = batch
                inputs = inputs.to(device).float()
                outputs = best_model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            # Calculate metrics
            f1_weighted = f1_score(all_labels, all_preds, average="weighted")
            f1_individual = f1_score(all_labels, all_preds, average="macro")
            precision = precision_score(all_labels, all_preds, average="weighted")
            recall = recall_score(all_labels, all_preds, average="weighted")
            conf_matrix = confusion_matrix(all_labels, all_preds)
            classification_rep = classification_report(all_labels, all_preds, output_dict=True)

            # Log metrics to wandb
            wandb.log({
                "test_f1_weighted": f1_weighted,
                "test_f1_individual": f1_individual,
                "test_precision": precision,
                "test_recall": recall,
                "confusion_matrix": conf_matrix,
                "classification_report": classification_rep
            })





# Run the sweep
# wandb.agent(sweep_id, function=train)


Create sweep with ID: gticx39q
Sweep URL: https://wandb.ai/finnhenri-smidt/Alzheimer-Detection/sweeps/gticx39q


In [3]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: sriih3lb with config:
wandb: 	batch_size: 160
wandb: 	ensemble_variant: simple
wandb: 	epochs: 15
wandb: 	learning_rate: 0.005
wandb: 	model_name: MobileVit-s
wandb: 	random_seed: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Seed set to 0
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name | Type   | Params
--------------------------------
0 | fc   | Linear | 6.4 K 
--------------------------------
6.4 K     Trainable params
0         Non-trainable params
6.4 K     Total params
0.026     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=80). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

In [1]:
import torch
from torchvision import transforms
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from transformers import MobileViTImageProcessor
import wandb

from dataset import MRIFeatureDataModule, MRIFeatureDataset
from models import SimpleEnsembleModel, AdvancedEnsembleModel
from utils import get_best_device, LoginCredentials

from datetime import datetime
import lightning.pytorch as pl
import torch
import numpy as np
import random
from sklearn.metrics import f1_score
best_model_class = SimpleEnsembleModel
best_model = best_model_class.load_from_checkpoint('model_checkpoints/ensemble_simple_MobileVit-s/lr_0.0001.ckpt',
    feature_size=1600,
    num_classes=4)

